### create a model to run through radlite
### model creates the (dust) temperature profile so no need to run radmc3d mctherm

In [1]:
import os, subprocess
import numpy as np
import matplotlib.pyplot as plt
from radmc3dPy import *
%matplotlib inline

Fast (Fortran90) Mie-scattering module could not be imported. Falling back to the slower Python version.


In [2]:
# model parameters
mstar = ['1.0*ms']        # stellar mass
tstar = [4000.0]          # Teff (K)
rstar = ['2.0*rs']        # stellar radius

mdisk = '1e-5*ms'         # dust disk mass
dusttogas = 0.001         # dust to gas ratio
rin = '0.1*au'            # inner disk radius
rdisk = '5*au'            # outer disk radius
gap_rin='[0.0*au]'        # gap inner radius
gap_rout='[0.0*au]'       # gap outer radius
Tmid = 100                # midplane temperature at 1 au
Tatm = 350                # atmospheric temperature at 1 au

In [3]:
keys = {'model':'ppdisk_jw', 'mstar':mstar, 'rstar':rstar, 'tstar':tstar,
        'mdisk':mdisk, 'dusttogas':dusttogas,
        'rin':rin, 'rdisk':rdisk,
        'xbound':'[0.05*au, 0.5*au, 5.0*au]', 'nx':'[150, 150]', 'nphot':1e6,
        'nz':'0', 'binary':False,
        'Tmid':Tmid, 'Tatm':Tatm, 'writeDustTemp':True}

In [4]:
# setup problem_params.log with default parameters
analyze.writeDefaultParfile('ppdisk_jw')

Writing problem_params.inp


In [5]:
# update the parameters in problem_params.inp and write out files in new radmc3d format
setup.problemSetupDust(**keys)

Writing problem_params.inp
Writing dustopac.inp
Writing wavelength_micron.inp
Writing amr_grid.inp
Writing stars.inp
-------------------------------------------------------------
Luminosities of radiation sources in the model :
Reading wavelength_micron.inp
As calculated from the input files :
Stars : 
  Star #0 + hotspot        : 3.564346e+33
Continuous starlike source : 0.000000e+00
 
-------------------------------------------------------------
Writing dust_density.inp
Writing dust_temperature.dat
Writing radmc3d.inp


In [6]:
# translate temperature array to old radmc format (settings keys to old does not do this unfortunately)
data = analyze.readData(dtemp=True)
Tdust = data.dusttemp[:,:,0,0]
if np.min(Tdust) == 0:
    print("WARNING: temperature array has zeros -- increase nphot")
nr, nt = Tdust.shape
fname = "dusttemp_final.dat"
with open(fname, "w") as wfile:
    print("Writing " + fname)
    wfile.write(f"   1    {nr:d}   {nt//2:d}   1\n")
    wfile.write(" \n")
    wfile.write("   1\n")
    for i in range(nr):
        for j in range(nt//2):
            wfile.write(f"{Tdust[i,j]:.7f}\n")

fname = "dusttemp.info"
with open(fname, "w") as wfile:
    print("Writing " + fname)
    wfile.write("  -2\n")
    wfile.write("   1\n")
    wfile.write("   1\n")
    wfile.write("   1\n")
    wfile.write("   1\n")

Reading amr_grid.inp
Reading wavelength_micron.inp
Reading dust_temperature.dat
Writing dusttemp_final.dat
Writing dusttemp.info


In [7]:
# make a directory for the radmc new format outputs
outputdir = "./radmc"
print(f"Moving radmc files to output directory {outputdir}")
if os.path.exists(outputdir):
    print("Will overwrite existing files")
else:
    print("Directory does not exist; will create")
    os.makedirs(outputdir)

Moving radmc files to output directory ./radmc_outputs_new_format
Directory does not exist; will create


In [8]:
filelist = ["problem_params.inp", "dustopac.inp", "wavelength_micron.inp", "amr_grid.inp", "stars.inp", \
            "dust_density.inp", "dust_temperature.dat", "radmc3d.inp", \
            "plot_radmc3d_model.ipynb"]
for file in filelist:
    os.system("mv "+file+" radmc/")

In [9]:
# create problem_params.inp, update parameters, and write out files for radlite)
# (note that radlite uses the old format of radmc3d)
analyze.writeDefaultParfile('ppdisk_jw')
setup.problemSetupDust(**keys, old=True)

# get rid of the new format dust temperature file which is not needed
print('Removing dust_temperature.dat')
os.remove('dust_temperature.dat')

Writing problem_params.inp
Writing problem_params.inp
Writing dustopac.inp
Writing frequency.inp
Reading dustkappa_silicate.inp
Writing dustopac_1.inp
Writing frequency.inp
Writing radius.inp
Writing theta.inp
Writing starinfo.inp
Writing starspectrum.inp
Writing dust_temperature.dat
Removing dust_temperature.dat


In [10]:
# make a directory for the radmc outputs for radlite to use
outputdir = "./radlite"
print(f"Moving radmc files to output directory {outputdir}")
if os.path.exists(outputdir):
    print("Will overwrite existing files")
else:
    print("Directory does not exist; will create")
    os.makedirs(outputdir)

Moving radmc files to output directory ./radmc_outputs
Directory does not exist; will create


In [11]:
filelist = ["problem_params.inp", "dustdens.inp", "dustopac.inp", "dusttemp_final.dat", "dusttemp.info", \
            "frequency.inp", "dustopac_1.inp", "radius.inp", "theta.inp", "starinfo.inp", "starspectrum.inp", "radmc.inp"]
for file in filelist:
    os.system("mv "+file+" radlite/")

# can now move the new format file for the dust over
os.system("mv dustkappa_silicate.inp radmc/")

In [12]:
# move files over to radmc_outputs for radlite to run
# (I tried and failed to hide any errors that come up if you run this a second time and the files have already been moved...)
filelist = ["run_radlite*.ipynb", "model*.json", "spectrum.json", "line.inp", "molecule_co.inp"]
for file in filelist:
    cmd = "mv "+file+" radlite/"
    x = os.system(cmd + "> /dev/null")
    if x==0:
        print(cmd)

mv run_radlite*.ipynb radmc_outputs/
mv model*.json radmc_outputs/
mv spectrum.json radmc_outputs/
mv line.inp radmc_outputs/
mv molecule_co.inp radmc_outputs/
